##🔹Gráfico de Pareto Usando o Dashboard IA/BI da Databricks🔹

Você já ouviu falar da Lei de Pareto? 🤔 Também conhecida como a regra 80/20, ela afirma que **80% dos resultados vêm de 20% das causas**. Isso é um divisor de águas no mundo dos negócios, ajudando a identificar onde está o maior impacto e permitindo focar esforços no que realmente importa.

<img src="Pareto Regra.png" alt="Descrição da imagem" width="300">

**💡 No meu caso:** Utilizei o **Databricks** para criar um **gráfico de Pareto**, analisando as vendas por empresa e descobrindo quais delas compõem os 80% do volume de vendas. Vou te mostrar agora o passo a passo dessa construção e as técnicas aplicadas, com destaque para o uso de **Window Functions**. 🚀

---
### **Passo a Passo da Query 🔍**
#### **1️⃣ Agregação das Vendas Totais**
Primeiro, agrupei as vendas totais de cada loja em um CTE chamado `TotalVendas`:
```
WITH TotalVendas AS (
    SELECT
        empresa,
        SUM(valor_vendas) AS total_vendas
    FROM workshop.gold.notas
    WHERE DATE(dt_referencia) BETWEEN '{{dt_referencia.start}}' AND '{{dt_referencia.end}}'
    GROUP BY empresa
),
```
#### **🔎 O que fiz aqui:**

* **Agrupei os dados:** Totalizei as vendas por empresa no intervalo de datas fornecido.
* Esse é o **primeiro passo** para identificar quais empresas mais contribuem para o volume total.
---

#### **2️⃣ Ordenação e Ranking das Lojas**
Em seguida, apliquei o CTE `RankedVendas` para ordenar as empresas e calcular valores percentuais e acumulados:
```
RankedVendas AS (
    SELECT
        empresa,
        total_vendas,
        RANK() OVER (ORDER BY total_vendas DESC) AS rank,
        total_vendas / SUM(total_vendas) OVER () AS pct_total,
        SUM(total_vendas) OVER (ORDER BY total_vendas DESC) AS vendas_acumuladas
    FROM TotalVendas
),
```
#### **🔎 Técnicas utilizadas:**

* **RANK():** Classifiquei as empresas pelo volume total de vendas.
* **% de Contribuição (pct_total):** Identifiquei a participação percentual de cada empresa.
* **Vendas Acumuladas:** Somei os valores de forma acumulativa.
---

#### **3️⃣ Cálculo da Porcentagem Acumulada**
No CTE `PctAcumulado`, calculei a porcentagem acumulada das vendas:
```
PctAcumulado AS (
    SELECT
        *,
        vendas_acumuladas / SUM(total_vendas) OVER () AS pct_acumulado
    FROM RankedVendas
)
```
#### **🔎 Aqui, respondi:**

- **Quem compõe os 80%?** A porcentagem acumulada identifica as empresas que mais impactam o volume total.
---

#### **4️⃣ Seleção Final e Filtro de Pareto**
Por fim, filtrei as empresas que compõem os **80% iniciais** das vendas:
```
SELECT
    empresa,
    ROUND(total_vendas, 0) AS total_vendas,
    ROUND(pct_acumulado * 100, 2) AS pct_acumulado
FROM PctAcumulado
WHERE IF(pct_acumulado <= 0.8, 'S', 'N') IN ({{flag_pareto}})
ORDER BY pct_acumulado
LIMIT {{top}}
```
#### **🔎 Finalizando:**

* **Filtro Pareto:** Seleciona apenas as empresas dentro dos 80% do total acumulado.
* **Exibição:** Limitei os resultados para mostrar apenas as lojas relevantes.
---

#### **Resultado Final 🎯**
* **1️⃣ Query Resultante:**
Aqui está a query completa que consolidou todas as etapas.
```
WITH TotalVendas AS (
    SELECT
        empresa,
        SUM(valor_vendas) AS total_vendas
    FROM workshop.gold.notas
    WHERE DATE(dt_referencia) BETWEEN '{{dt_referencia.start}}' AND '{{dt_referencia.end}}'
    GROUP BY empresa
),

RankedVendas AS (
    SELECT
        empresa,
        total_vendas,
        RANK() OVER (ORDER BY total_vendas DESC) AS rank,
        total_vendas / SUM(total_vendas) OVER () AS pct_total,
        SUM(total_vendas) OVER (ORDER BY total_vendas DESC) AS vendas_acumuladas
    FROM TotalVendas
),

PctAcumulado AS (
    SELECT
        *,
        vendas_acumuladas / SUM(total_vendas) OVER () AS pct_acumulado
    FROM RankedVendas
)

SELECT
    empresa,
    ROUND(total_vendas, 0) AS total_vendas,
    ROUND(pct_acumulado * 100, 2) AS pct_acumulado
FROM PctAcumulado
WHERE IF(pct_acumulado <= 0.8, 'S', 'N') IN ({{flag_pareto}})
ORDER BY pct_acumulado
LIMIT {{top}}
```

* **2️⃣ Gráfico Interativo:**
O gráfico criado no **Databricks** permite a visualização interativa dos resultados, com filtros ajustáveis por parâmetros. 🖥️📊

**Resultado das Querys:**

<img src="Pareto Resultado.png" alt="Descrição da imagem" width="1000">

**Aplicação em um dashboard exemplo:**

<img src="Exemplo aplicação pareto.png" alt="Descrição da imagem" width="1000">

* **3️⃣ Demonstração em Vídeo:**
Confira o vídeo explorando o gráfico e extraindo **insights valiosos** dos dados.
👉 [Assista agora!](https://drive.google.com/file/d/1qAPaVoK7i7NGY1UPA7cdtuhNZd_aJK_G/view?usp=drive_link)

---

#### **Conclusão 💡**
Este gráfico de Pareto no **Databricks** é uma poderosa ferramenta para identificar **onde concentrar esforços** e entender quais clientes ou empresas fazem a diferença no resultado.

**📌Recursos e Técnicas Utilizados:**

- **Conceito da Lei de Pareto (Regra 80/20):** Para priorizar os itens de maior impacto.
- **Funções SQL Aplicadas:**
    - **Window Functions:** Para calcular rankings, porcentagens e valores acumulados.
    - **RANK() e OVER():** Para criar rankings e acumular vendas em ordem decrescente.
    - **WITH():** Para organizar a query em etapas claras e reutilizáveis.
    - **Parâmetros Dinâmicos ({{parametro_filtro}}):** Para flexibilidade nos filtros e configurações do dashboard.

Espero que este conteúdo seja útil e inspire suas próximas análises!😉